In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 20337531
paper_name = 'dias_sa_correia_2010' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [5]:
original_data1 = pd.read_csv('raw_data/TableS1.txt', header=None, names=['genes','orfs'], sep='\t')
original_data2 = pd.read_csv('raw_data/TableS2.txt', header=None, names=['genes','orfs'], sep='\t')

In [6]:
print('Original data dimensions: %d x %d' % (original_data1.shape))
print('Original data dimensions: %d x %d' % (original_data2.shape))

Original data dimensions: 288 x 2
Original data dimensions: 453 x 2


In [7]:
original_data1['orfs'] = original_data1['orfs'].astype(str)
original_data2['orfs'] = original_data2['orfs'].astype(str)

In [8]:
# Eliminate all white spaces & capitalize
original_data1['orfs'] = clean_orf(original_data1['orfs'])
original_data2['orfs'] = clean_orf(original_data2['orfs'])

In [9]:
# Translate to ORFs 
original_data1['orfs'] = translate_sc(original_data1['orfs'], to='orf')
original_data2['orfs'] = translate_sc(original_data2['orfs'], to='orf')

In [10]:
# Make sure everything translated ok
t = looks_like_orf(original_data1['orfs'])
print(original_data1.loc[~t,])

Empty DataFrame
Columns: [genes, orfs]
Index: []


In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data2['orfs'])
print(original_data2.loc[~t,])

Empty DataFrame
Columns: [genes, orfs]
Index: []


In [12]:
original_data1.set_index('orfs', inplace=True)
original_data2.set_index('orfs', inplace=True)
original_data1.index.name='orf'
original_data2.index.name='orf'

In [13]:
# High-susceptibility
original_data1['data'] = -2

# Moderate-susceptibility
original_data2['data'] = -1

In [14]:
original_data1 = original_data1[['data']].copy()
original_data2 = original_data2[['data']].copy()

In [15]:
original_data1 = original_data1.groupby(original_data1.index).mean()
original_data2 = original_data2.groupby(original_data2.index).mean()

In [16]:
original_data1.shape

(286, 1)

In [17]:
original_data2.shape

(453, 1)

In [18]:
original_data = original_data1.join(original_data2, lsuffix='_high', rsuffix='_mod', how='outer')

In [19]:
original_data['data'] = original_data[['data_high','data_mod']].mean(axis=1)

In [20]:
original_data.drop(columns=['data_high','data_mod'], inplace=True)

In [21]:
original_data.shape

(739, 1)

# Load & process tested strains

In [22]:
# tested = pd.read_excel('raw_data/List of strains tested.xlsx', sheet_name='Tabelle2')

In [23]:
# tested.head()

In [24]:
# tested['orf'] = tested['ORF'].astype(str)

In [25]:
# tested['orf'] = clean_orf(tested['orf'])

In [26]:
# tested['orf'] = translate_sc(tested['orf'], to='orf')

In [27]:
# # Make sure everything translated ok
# t = looks_like_orf(tested['orf'])
# print(tested.loc[~t,:])

In [28]:
# tested_orfs = tested['orf'].unique()

In [29]:
# missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
# missing

In [30]:
# # Remove missing (the data list contains HAP and HET screen results, so some of the genes are likely to be essential)
# original_data = original_data.reindex(index=tested_orfs, fill_value=0)

In [31]:
# original_data.shape

# Prepare the final dataset

In [32]:
data = original_data.copy()

In [33]:
dataset_ids = [16607]
datasets = datasets.reindex(index=dataset_ids)

In [34]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [35]:
data.head()

dataset_id,16607
data_type,value
orf,
YAL002W,-2.0
YAL004W,-1.0
YAL005C,-1.0
YAL021C,-2.0
YAL024C,-2.0


## Subset to the genes currently in SGD

In [36]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [37]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16607
,data_type,value
gene_id,orf,
2,YAL002W,-2.0
1863,YAL004W,-1.0
4,YAL005C,-1.0
19,YAL021C,-2.0
22,YAL024C,-2.0


# Normalize

In [38]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [39]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [40]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      16607         
data_type       value   valuez
gene_id orf                   
2       YAL002W  -2.0 -3.41348
1863    YAL004W  -1.0 -1.70674
4       YAL005C  -1.0 -1.70674
19      YAL021C  -2.0 -3.41348
22      YAL024C  -2.0 -3.41348

# Print out

In [41]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [45]:
# from IO.save_data_to_db3 import *

In [46]:
# save_data_to_db(data_all, paper_pmid)